In [1]:
# ###########  implementation of a convnet on the cifar-10 dataset ############################################
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from utilities import load_images
from sklearn.model_selection import train_test_split

In [2]:
data=load_images()

In [3]:
def weights_variable(shape):
    weights=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(weights)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding="SAME")

def max_pool2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

def conv_layer(inpu,shape):
    weights=weights_variable(shape)
    bias=bias_variable([shape[3]])
    return tf.nn.relu(conv2d(inpu,weights)+bias)

def full_layer(inp,size):
    ln_size=int(inp.get_shape()[1])
    weights=weights_variable([ln_size,size])
    bias=bias_variable([size])
    return tf.matmul(inp,weights)+bias

In [8]:
c1,c2,c3=30,50,80
f1=500
classification=tf.Graph()
with classification.as_default():
    with tf.name_scope("placeholders"):
        x=tf.placeholder(tf.float32,shape=(None,28*28))
        y=tf.placeholder(tf.int32,shape=(None,))
        drop=tf.placeholder(tf.float32)
        
    with tf.name_scope("transformation"):
        x_img=tf.reshape(x,[-1,28,28,1])
        
    with tf.name_scope("conv_1"):
        conv1_1=conv_layer(x_img,shape=[3,3,1,c1])
        conv1_2=conv_layer(conv1_1,shape=[3,3,c1,c1])
        conv1_3=conv_layer(conv1_2,shape=[3,3,c1,c1])
        conv1_pool=max_pool2x2(conv1_1)
        
    with tf.name_scope("conv2"):
        conv2_1=conv_layer(conv1_pool,[3,3,c1,c2])
        conv2_2=conv_layer(conv2_1,[3,3,c2,c2])
        conv2_3=conv_layer(conv2_2,[3,3,c2,c2])
        conv2_pool=max_pool2x2(conv2_1)
        
    with tf.name_scope("conv3"):
        conv3_1=conv_layer(conv2_pool,[3,3,c2,c3])
        conv3_2=conv_layer(conv3_1,[3,3,c3,c3])
        conv3_3=conv_layer(conv3_2,[3,3,c3,c3])
        conv3_pool=tf.nn.max_pool(conv3_1,ksize=[1,7,7,1],strides=[1,7,7,1],padding="SAME")
        flat_out=tf.reshape(conv3_pool,[-1,c3])
        
    with tf.name_scope("hidden"):
        hidden1=tf.nn.dropout(full_layer(flat_out,f1),drop)
        prediction=full_layer(hidden1,10)
    
    with tf.name_scope("train_step"):
        xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=prediction,labels=y)
        train_loss=tf.reduce_mean(xentropy)
        train_op=tf.train.AdamOptimizer(0.01).minimize(train_loss)
        
    with tf.name_scope("saver"):
        saver=tf.train.Saver()

In [5]:
dataset=data['data'].reshape(-1,28*28)
x_train,x_test,y_train,y_test=train_test_split(dataset,data['targets'])
x_train,x_test,y_train,y_test=np.array(x_train),np.array(x_test),np.array(y_train),np.array(y_test)

def percent(rec,exp):
    import sys
    percent=str(round((rec/exp)*100,2))+"%"
    sys.stdout.write(percent)
    sys.stdout.write("\r")

In [ ]:
n_epochs=20
batch_size=1000
N=x_train.shape[0]
with classification.as_default():
    with tf.Session() as sess:
#         sess.run(tf.global_variables_initializer())
        saver.restore(sess,r"/temp/tf_models/convolutions_temp.ckpt")
        step=0
        for epoch in range(n_epochs):
#             print("epoch:{}".format(epoch))
            pos=0
            while pos<N:
                saver.save(sess,r"/temp/tf_models/convolutions_temp.ckpt")
                percent(pos,N)
                x_batch=x_train[pos:pos+batch_size]
                y_batch=y_train[pos:pos+batch_size]
                loss,_=sess.run([train_loss,train_op],feed_dict={x:x_batch,y:y_batch,drop:0.5})
                pos+=batch_size
            print("epoch{}:{}".format(epoch,loss)) 
        print("done     ")

INFO:tensorflow:Restoring parameters from /temp/tf_models/convolutions_temp.ckpt
epoch0:0.3014345169067383
epoch1:0.10703881084918976
epoch2:0.11128994822502136
epoch3:0.02442348562180996
epoch4:0.014204118400812149
epoch5:0.015037823468446732
epoch6:0.014359831809997559
epoch7:0.009290809743106365
epoch8:0.008106707595288754
epoch9:0.015886561945080757
epoch10:0.010554688982665539
epoch11:0.010190696455538273


In [6]:
train_batch=100
with classification.as_default():
    with tf.Session() as sess:
        saver.restore(sess,r"/temp/tf_models/convolutions_temp.ckpt")
        for name,train,y in [["test_score",x_test,y_test],["train_score",x_train,y_train]]:
            pos=0
            out=[]
            out=np.array(sess.run([prediction],feed_dict={x:train[pos:pos+train_batch],drop:1.})).reshape(-1,10)
            while pos<train.shape[0]:
                pos+=train_batch
                temp=np.array(sess.run([prediction],feed_dict={x:train[pos:pos+train_batch],drop:1.})).reshape(-1,10) 
                out=np.vstack([out,temp])
                percent(pos,train.shape[0])
            print(name,np.array(np.argmax(out,1)==y).mean())

INFO:tensorflow:Restoring parameters from /temp/tf_models/convolutions_temp.ckpt
test_score 0.960478530229
train_score 0.959766431674


In [71]:
with classification.as_default():
    with tf.Session() as sess:
        saver.restore(sess,r"/temp/tf_models/convolutions_temp.ckpt")
#         sess.run(tf.global_variables_initializer())
        data=sess.run([prediction],feed_dict={x:x_train[:100],y:y_train[:10],drop:1.})

INFO:tensorflow:Restoring parameters from /temp/tf_models/convolutions_temp.ckpt
